In [1]:
import os
import torch
import numpy as np
import random
import pandas as pd


Parameters

In [2]:
num_targets = 500
num_items = 100

num_instances = 100
test_pct = 0.2
num_random_iter = 30



Import of User data 

In [9]:
df_user = pd.read_pickle("../data/weibo/userProfile.pkl")
df_user.index = df_user['id']
def featvec(feat, clist):
    """
    onehot encoding
    """
    output = [0] * len(clist)
    output[clist.index(feat)] = 1
    return output

DATE_VAR = ['created_at']
NUM_VAR = ['bi_followers_count', 'followers_count', 'friends_count', 'statuses_count']
CAT_VAR = ['city', 'verified', 'province', 'statuses_count', 'verified_type']
df_user_num = df_user[NUM_VAR]
df_user_num

,bi_followers_count,followers_count,friends_count,statuses_count
id,,,,
1657151084,0,33,162,0
1657149732,0,16,27,11
1657148500,4,79,29,25
1657146942,1,14,11,3
1657150271,73,390,151,24
...,...,...,...,...
1837213080,91,880,130,1713
1837246162,15,246,230,915
1837251604,42,472,169,979


In [12]:

'''
make link probability matrix
'''

f = open('ml-100k/u.data')
data = f.readlines()
f.close()

data = np.array([list(map(int, d.split()[:-1])) for d in data])
P_full = np.zeros([np.max(data[:,1]) , np.max(data[:,0])])
for d in data: 
    P_full[d[1]-1, d[0]-1] = .02 * d[2] 
P_full.shape

(1682, 943)

In [ ]:


'''
make features
'''
num_movies, num_movie_features = item_df.shape
num_users, num_user_features = user_df.shape
num_features = num_movie_features + num_user_features

features = np.zeros([num_movies, num_users, num_features])

for u in range(num_users):
    features[:, u, :] = np.c_[np.tile(user_df.iloc[u],(num_movies, 1)), np.array(item_df)]



In [ ]:

'''
make random instances
'''

Ps   = np.zeros([num_instances, num_items, num_targets])
data = np.zeros([num_instances, num_items, num_targets, num_features])
for t in range(num_instances):
    m_idx = random.sample(range(np.shape(P_full)[1]), num_items)
    u_idx = random.sample(range(np.shape(P_full)[1]), num_targets)
    P_used  = P_full[m_idx,:]
    Ps[t] = P_used[:, u_idx]
    data[t] = features[m_idx, :, :][:, u_idx, :]



In [ ]:

path = 'instances/'
if os.path.exists(path + 'Ps.npz') == False:
    np.savez(path + 'Ps.npz', Ps = Ps)
if os.path.exists(path + 'data.npz') == False:
    np.savez(path + 'data.npz', data = data)

for i in range(num_random_iter):
    test = random.sample(range(num_instances), int(test_pct*num_instances))
    train = [i for i in range(num_instances) if i not in test]
    if os.path.exists(path + '{}.npz'.format(i)) == False:
        np.savez(path + '{}'.format(i), train = train, test = test)
